In [0]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

Definition from the book: (4.5.2)

$y_i = f(x_i)$, $x_i\in[a,b]$

$S_{3,i}(x)=a_{0,i} + a_{1,i}(x-x_i) + a_{2,i}(x-x_i)^2 + a_{3,i}(x-x_i)^3$, $x \in [x_i, x_{i+1}]$ (4.68 from the book)

$S_{3}(x) \in C_2[a,b]$ 

Book page: 188

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

Functional compatibility conditions (4.69 from the book):

$\delta S_{3,i}(x)|_{x=x_i} = S_{3,i}(x_i)-f(x_i) = 0$,

$\delta S_{3,i}(x)|_{x=x_{i+1}} = S_{3,i}(x_{i+1})-f(x_{i+1}) = 0$;

Differential compatibility conditions (4.70 from the book):

$\delta S_{3,i}^{''}(x)|_{x=x_i} = S_{3,i}^{''}(x_i)-f^{''}(x_i) = 0$,

$\delta S_{3,i}^{''}(x)|_{x=x_{i+1}} = S_{3,i}^{''}(x_{i+1})-f^{''}(x_{i+1}) = 0$.

Order:
$p_1=\{0; 2\}$

Book page: 188

### 3. Formulate stitching conditions [mathematical formula]

After the 4.71 formula in the book, we have condition of continuity:
$S_{3,i-1}^{(p_1)}(x)|_{x=x_i} = S_{3,i}^{(p_1)}(x)|_{x=x_i}, i = \overline{1, n-1}$

Spline second derivatives: $S_{3,i-1}^{'}(x)|_{x=x_i} = S_{3,i}^{'}(x)|_{x=x_i}, i = \overline{1, n-1}$

(for all internal nodes)

Book page: 189

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

These conditions in result provide the same slope and change of the function (stitching condition).  
=> derivative of the function is smooth, then we understand that function is smooth. 

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

$\begin{cases} 
  \Delta S_{3,i}(x)|_{x=x_i} = S_{3,i}(x_i)-f(x_i) = 0 \\
  \Delta S_{3,i}(x)|_{x=x_{i+1}} = S_{3,i}(x_{i+1})-f(x_{i+1}) = 0 \\
  \Delta S_{3,i}^{''}(x)|_{x=x_i} = S_{3,i}^{''}(x_i)-f^{''}(x_i) = 0 \\
  \Delta S_{3,i}^{''}(x)|_{x=x_{i+1}} = S_{3,i}^{''}(x_{i+1})-f^{''}(x_{i+1}) = 0. \\
   m_i = f''(x_{i}) \\
  \Delta m_i = m_{i+1} - m_i \\
  i = \overline{1, n-1}
\end{cases}$

$\begin{cases} 
  S_{3,i}(x)=a_{0,i} + a_{1,i}(x-x_i) + a_{2,i}(x-x_i)^2 + a_{3,i}(x-x_i)^3\\
  S_{3,i}^{'}(x)=a_{1,i}+2a_{2,i}(x-x_i)+3a_{3,i}(x-x_i)^2\\
  S_{3,i}^{''}(x)=2(a_{2,i}+3a_{3,i}(x-x_i))
\end{cases}$

$\begin{cases} 
    a_{0,i} + a_{1,i}(x_i-x_i) + a_{2,i}(x_i-x_i)^2 + a_{3,i}(x_i-x_i)^3 - f(x_i) = 0 \\
    a_{0,i} + a_{1,i}(x_{i+1}-x_i) + a_{2,i}(x_{i+1}-x_{i})^2 + a_{3,i}(x_{i+1}-x_i)^3 - f(x_{i+1}) = 0 \\
    2a_{2,i}+6a_{3,i}(x_i-x_i)-f^{''}(x_i) = 0 \\
    2a_{2,i}+6a_{3,i}(x_{i+1}-x_{i})-f^{''}(x_{i+1}) = 0
\end{cases}$

$\begin{cases} 
    a_{0,i} = f(x_i) \\
    a_{0,i} + a_{1,i}(x_{i+1}-x_i) + a_{2,i}(x_{i+1}-x_{i})^2 + a_{3,i}(x_{i+1}-x_i)^3 - f(x_{i+1}) = 0 \\
    a_{2,i} = f^{''}(x_i) / 2\\
    2a_{2,i}+6a_{3,i}(x_{i+1}-x_{i})-f^{''}(x_{i+1}) = 0
\end{cases}$

$h_{i+1}=x_{i+1}-x_i$

$\begin{cases} 
  a_{0,i} = f(x_i) \\
  a_{2,i} = \frac{m_i}{2} \\
  a_{3,i}=\frac{\Delta m_i}{6h_{i+1}}
\end{cases}$


$f(x_i) + a_{1,i}h_{i+1} + \frac{m_i}{2}h_{i+1}^2 + \frac{\Delta m_i}{6(h_{i+1})}h_{i+1}^3 = f(x_{i+1}) $

$a_{1,i} = \frac{\Delta f_i - \frac{m_i}{2}h_{i+1}^2 - \frac{\Delta m_i}{6(h_{i+1})}h_{i+1}^3}{h_{i+1}} = \frac{\Delta f_i}{h_{i+1}} - \frac{m_i  h_{i+1}}{2} - \frac{\Delta m_i  h_{i+1}}{6}$


$S'_{3, i}(x)=a_{1,i}+2a_{2,i}(x_{i+1}-x_{i})+3a_{3,i}(x_{i+1}-x_{i})^2$

$S'_{3, i-1}(x)=a_{1,i-1}+2a_{2,i-1}(x_{i+1}-x_{i})+3a_{3,i-1}(x_{i+1}-x_{i})^2$

$S'_{3, i}(x)|_{x=x_i} = S'_{3, i-1}(x)|_{x=x_i}$

$a_{1,i}=a_{1,i-1}+2a_{2,i-1}h_{i}+3a_{3,i-1}h_{i}^2$

$\frac{\Delta f_i}{h_{i+1}} - \frac{m_i  h_{i+1}}{2} - \frac{\Delta m_i h_{i+1}}{6} = \frac{\Delta f_{i-1}}{h_{i}} - \frac{m_{i-1}  h_{i}}{2} - \frac{\Delta m_{i-1}  h_{i}}{6} + \frac {2 m_{i-1} h_{i}}{2} +\frac{3\Delta m_{i-1}h_{i}^2}{6h_{i}} $

.

$\frac{m_{i-1}h_i}{6} + \frac{m_i(h_i+h_{i+1})}{3} + \frac{m_{i+1}h_{i+1}}{6} = \frac{\Delta f_{i}}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}}$

Result: $\frac{\Delta f_{i}}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}}$


### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

$$
\begin{pmatrix}
1 & \frac{h2}{2(h_1+h_2)} & 0 & 0 & ...  & 0 & 0 \\
\frac{h_2}{6} & \frac{(h_2+h_3)}{3} & \frac{h_3}{6} & 0 & ... & 0 & 0  \\
0 & \frac{h_3}{6} & \frac{h_3+h_4}{3} & \frac{h_4}{6} & ... & 0 & 0 \\
&... & ... & ... & ... & ...  \\
0 & 0 & 0 & 0 & ... & \frac{h_{n-1}}{6} & \frac{h_{n-1}+h_{n}}{3} \\ 
\end{pmatrix}
\quad
\begin{pmatrix}
m_0 \\
m_1 \\
m_2 \\
... \\
m_{n-1} \\
m_{n}
\end{pmatrix}=
\begin{pmatrix}
\frac{\Delta f_1}{h_{2}}-\frac{\Delta f_{0}}{h_1} \\
\frac{\Delta f_2}{h_{3}}-\frac{\Delta f_{1}}{h_2} \\
\frac{\Delta f_3}{h_{4}}-\frac{\Delta f_{2}}{h_3} \\
... \\
\frac{\Delta f_{n-2}}{h_{n-1}}-\frac{\Delta f_{n-3}}{h_{n-2}} \\
\frac{\Delta f_{n-1}}{h_{n}}-\frac{\Delta f_{n-2}}{h_{n-1}}
\end{pmatrix}
$$

Book page: 31

### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

Unknown variable is $m_i$, $i \in [a, b]$. System is not closed.

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

$$
\begin{pmatrix}
1 & \frac{h2}{2(h_1+h_2)} & 0 & 0 & ...  & 0 & 0 \\
\frac{h_2}{6} & \frac{(h_2+h_3)}{3} & \frac{h_3}{6} & 0 & ... & 0 & 0  \\
0 & \frac{h_3}{6} & \frac{h_3+h_4}{3} & \frac{h_4}{6} & ... & 0 & 0 \\
&... & ... & ... & ... & ...  \\
0 & 0 & 0 & 0 & ... & \frac{h_{n-1}}{6} & \frac{h_{n-1}+h_{n}}{3} \\ 
\end{pmatrix}
\quad
\begin{pmatrix}
m_0 \\
m_1 \\
m_2 \\
... \\
m_{n-1} \\
m_{n}
\end{pmatrix}=
\begin{pmatrix}
\frac{\Delta f_1}{h_{2}}-\frac{\Delta f_{0}}{h_1} \\
\frac{\Delta f_2}{h_{3}}-\frac{\Delta f_{1}}{h_2} \\
\frac{\Delta f_3}{h_{4}}-\frac{\Delta f_{2}}{h_3} \\
... \\
\frac{\Delta f_{n-2}}{h_{n-1}}-\frac{\Delta f_{n-3}}{h_{n-2}} \\
\frac{\Delta f_{n-1}}{h_{n}}-\frac{\Delta f_{n-2}}{h_{n-1}}
\end{pmatrix}
$$


$$
\begin{pmatrix}
1 & -P1 & 0 & 0 & ...  & 0 & 0 & 0 \\
0 & 1 & -P2 & 0 & ... & 0 & 0 & 0 \\
0 & 0 & 1 & -P3 & ... & 0 & 0 & 0  \\
&... & ... & ... & ... & ... & ... \\
0 & 0 & 0 & 0 & ... & 0 & 1 & -P_{n-1} \\ 
0 & 0 & 0 & 0 & ... & 0 & 0 & 1 \\
\end{pmatrix}
\quad
\begin{pmatrix}
m_0 \\
m_1 \\
m_2 \\
... \\
m_{n-1} \\
m_{n}
\end{pmatrix}=
\begin{pmatrix}
Q_1 \\
Q_2 \\
Q_3 \\
... \\
Q_{n-1} \\
Q_{n}
\end{pmatrix}
$$

Book page: 31

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Forward:

$x_{i-1} = P_{i-1}x_i+Q_{i-1}$, where $i = \overline{1, n-1}$

$\frac{(P_{i-1}x_i + Q_{i-1}) h_i}{6} - \frac{x_i(h_i+h_{i+1})}{3} + \frac{x_{i+1}h_{i+1}}{6} = \frac{\Delta f_{i}}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}}$

$a_i = \frac{h_i}{6}\\
b_i = \frac{h_i + h_{i+1}}{3}\\
c_i = \frac{h_{i+1}}{6}\\\
q = \frac{\Delta f_{i}}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}}$

$x_i = -\frac{c_i x_{i+1}}{a_i P_{i-1} - b_i} - \frac{a_i Q_{i-1}}{a_i P_{i-1} - b_i} + \frac{q_i}{a_i P_{i-1} - b_i} = \frac{c_i x_{i+1}}{b_i - a_i P_{i-1}} + \frac{a_i Q_{i-1} - q_i}{b_i - a_i P_{i-1}}$

$\frac{c_i x_{i+1}}{b_i - a_i P_{i-1}} + \frac{a_i Q_{i-1} - q_i}{b_i - a_i P_{i-1}} = P_{i}x_{i+1}+Q_{i}$

$P_{i} = \frac{c_i }{b_i - a_i P_{i-1}}$

$Q_i = \frac{a_i Q_{i-1} - q_i}{b_i - a_i P_{i-1}}$

Reverse:

$x_{n-1} = P_{n-1}x_n + Q_{n-1}$

$\begin{cases}
  q_n = a_n x_{n-1} - b_n x_n \\
  q_n = a_n(P_{n-1}x_n + Q_{n-1}) - b_n x_n 
\end{cases}$

$x_n(a_n P_{n-1} - b_n) = q_n - a_n Q_{n-1} $

$x_n = Q_n = \frac{a_n Q_{n-1} - q_n}{b_n - a_n P_{n-1}}$


### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [0]:
def build_spline(x, y):
  pass

def compute_spline_value(spline):
  pass

def direct_pass():
  pass

def reverse_pass():
  pass

def main():
  pass


### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

$|| f^{(p)}(x)-S_3^{(p)}(x)||_{C[a,b]} = \max\limits_{[a,b]}|f^{(p)}(x)-S_3^{(p)}(x)|\leq M_4h^{4-p}$, $p=0,1,2$

Book page: 192, formula: 4.78

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

O(n) for forward, O(n) for reverse

=> result: O(n)



### Congrats!